In [86]:
LINE_LIMIT = 10000

In [90]:
tmp = !pwd
current_path = tmp[0]

import psycopg2
from psycopg2 import Error
import timeit

conn = ('kevin', '', '127.0.0.1', '5432', 'flightly')

class PostgresQuery(object):
    def __init__(self, query):
        user, password, host, port, database = conn
        self._query = query
        try:
            self._connection = psycopg2.connect(
                user = user,
                password = password,
                host = host,
                port = port,
                database = database
            )
            self._cursor = self._connection.cursor()
        except:
            print("Failed to connect to PostgreSQL DB")

    def query(self, times=1):
        try:
            results = self._run_sql()
            time = timeit.timeit(self._run_sql, number=times)
            self._connection.commit()
            return results, time
        except (Exception, psycopg2.DatabaseError) as error :
            print ("Error while executing SQL command", error)
        finally:
            self._cursor.close()
            self._connection.close()
    
    def _run_sql(self):      
        self._cursor.execute(self._query)
        try:
            return self._cursor.fetchall()
        except:
            return []

In [102]:
init_queries = {
    'drop_all_tables': """
        DROP SCHEMA public CASCADE;
        CREATE SCHEMA public;
        
        GRANT ALL ON SCHEMA public TO postgres;
        GRANT ALL ON SCHEMA public TO public;
        """,
    'create_airlines_table': """
        CREATE TABLE IF NOT EXISTS airlines (
          iata text,
          name text);
        """,
    'create_airports_table': """
        CREATE TABLE IF NOT EXISTS airports (
          iata text,
          name text,
          city text,
          state text,
          country text,
          latitude text,
          longitude text
          );
        """,
    'create_flights_table': """
        CREATE TABLE IF NOT EXISTS flights (
            year text,
            month text,
            day text,
            day_of_week text,
            airline text,
            flight_number text,
            tail_number text,
            origin_airport text,
            destination_airport text,
            scheduled_departure text,
            departure_time text,
            departure_delay text,
            taxi_out text,
            wheels_off text,
            scheduled_time text,
            elapsed_time text,
            air_time text,
            distance text,
            wheels_on text,
            taxi_in text,
            scheduled_arrival text,
            arrival_time text,
            arrival_delay text,
            diverted text,
            cancelled text,
            cancellation_reason text,
            air_system_delay text,
            security_delay text,
            airline_delay text,
            late_aircraft_delay text,
            weather_delay text
        );
        """,
    'load_airlines_csv': f"""
        COPY airlines
        FROM '{current_path}/csv/airlines.csv' DELIMITER ',' HEADER CSV;
        """,
    'load_airports_csv': f"""
        COPY airports
        FROM '{current_path}/csv/airports.csv' DELIMITER ',' HEADER CSV;
        """,
    'load_flights_csv': f"""
        COPY flights
        FROM PROGRAM 'head -{LINE_LIMIT} {current_path}/csv/flights.csv' DELIMITER ',' HEADER CSV;
        """
}

for query in init_queries.items():
    PostgresQuery(query[1]).query(0)

In [103]:
query, time = PostgresQuery("SELECT tail_number from flights where origin_airport = 'ATL' and airline = 'AA'").query(200)
time

0.7718273100017541

In [106]:
results, time = PostgresQuery("SELECT * from flights where tail_number = 'N129DL'").query(200)

print(time)
[record for record in results]

0.6763240820000647


[('2015',
  '1',
  '1',
  '4',
  'DL',
  '1680',
  'N129DL',
  'ATL',
  'SFO',
  '1040',
  '1043',
  '3',
  '17',
  '1100',
  '330',
  '302',
  '277',
  '2139',
  '1237',
  '8',
  '1310',
  '1245',
  '-25',
  '0',
  '0',
  None,
  None,
  None,
  None,
  None,
  None),
 ('2015',
  '1',
  '1',
  '4',
  'DL',
  '61',
  'N129DL',
  'SFO',
  'ATL',
  '1410',
  '1410',
  '0',
  '12',
  '1422',
  '269',
  '258',
  '239',
  '2139',
  '2121',
  '7',
  '2139',
  '2128',
  '-11',
  '0',
  '0',
  None,
  None,
  None,
  None,
  None,
  None)]